## Activation Functions

```text

```

In [ ]:
# Built-in library
from typing import Any, Optional, Sequence, Union

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Configure the backend
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
import seaborn as sns

# Custom import
from src.utilities import create_iris_data, smooth, load_data
from src.preprocessor import Standardizer, Normalizer


# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
fp = "../../data/wine_data.csv"

data = load_data(filename=fp)
data.head()


#### Preprocess Data

```text
- Standardize/normalize the data.
- Binarize the output.
- Split the data using hold out CV.
- Create PyTorch datasets and dataloader objects.
- Create a custom class that supports batch normalization.
- Train and evaluate the model.
- Visualize the model results.
```

In [ ]:
# Select all the features bar the target.
X = data.iloc[:, :-1]

# Normalize the data
norm_ = Normalizer(min_value=0, max_value=1)
X_norm = norm_.fit_transform(X=X)

X_norm.head()


In [ ]:
# Binarize output
choice_list = [0, 1]

# 3, 4, and 5 : 0, else 1
data["target"] = np.where(data["quality"].isin(np.arange(3, 6)), choice_list[0], choice_list[1])

data[["quality", "target"]].sample(5, random_state=5)

In [ ]:
# Convert to Tensors
X = torch.Tensor(X_norm.to_numpy())

# Requred by PyTorch to be 2D instead of 1D
y = torch.Tensor(data["target"].to_numpy()).reshape(-1, 1)

# Split data
TEST_SIZE, RANDOM_STATE = 0.2, 123

X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

X_train.shape, X_validation.shape


In [ ]:
BATCH_SIZE, SHUFFLE = 16, True
DROP_LAST = True


# Create PyTorch datasets and dataloader objects.
train_data = TensorDataset(X_train, y_train)
validation_data = TensorDataset(X_validation, y_validation)

train_DL = DataLoader(
    dataset=train_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE, drop_last=DROP_LAST
)
validation_DL = DataLoader(
    dataset=validation_data,
    batch_size=validation_data.tensors[0].shape[0],
)
